**Текущие задачи**:
- Реализация опроса новых данных о свежих статьях
- Описать `requirements` или сделать Docker-образ
- Выгружать датасет из памяти после того, как он был считан. Составлять новый датасет, выводить в csv и конкатенировать сами csv.

# Парсер

Реализованы парсеры данных, собраны тексты и метаинформация статей следующих ресурсов:
- Хабрахабр https://habr.com/
- Типичный программист https://tproger.ru/
- DOU
- Библиотека программиста
- Код
- Русскоязычные туториалы DigitalOcean


Также реализован механизм докачивания новых статей, чтобы учесть появление новых технологий. Чтобы не создавать повышенную нагрузку на серверы ресурсов и не вызывать их подозрений, опрашивание ресурсов происходит поочередно. Методы парсеров содержатся внутри класса `Parser`, методы оркестровки (планирование и общий пайплайн вызова парсеров) – в классе `ParserComposer`.

Системные требования:
- Каталог с датасетами
- Дравйвер `/usr/bin/chromedriver`

In [2]:
import pandas as pd

In [5]:
# standard library
import random
import datetime
import pickle
import os
import re
import uuid

# conections
import requests
from requests.exceptions import ConnectionError

# data science 
import pandas as pd
from tqdm.notebook import tqdm, trange

sources = {
    'tproger':
    {
        'base_url': 'https://tproger.ru/',
        'first_page_url': 'https://tproger.ru/page/1/',
        'find_all_args': ('a', {'class':'article-link'})
    },
    'habr':
    {
        'base_url': 'https://habr.com/',
        'first_page_url': 'https://habr.com/ru/page1/',
        'find_all_args': ('a', {'class':'post__title_link'})
    },
#     'proglib':
#     {
#         'base_url': 'https://proglib.io/',
#         'first_page_url': 'https://proglib.io/',
#         'btn_class': 'load-more active',
#     }
}

proxies = {
    'http': 'socks5h://127.0.0.1:9050',
    'https': 'socks5h://127.0.0.1:9050'
}


DATASETS_PATH = '/home/leo/DATASETS'

In [6]:
def k_to_num(s):
    if type(s) is str:
        s = s.replace(',', '.')
        if 'k' in s:
            s = s.replace('k', 'e3')
    return round(float(s))

class Parser:
    def __init__(self, source_name):
        self.source_name = source_name
        self.source = sources[source_name]
        self.df = pd.read_csv(f"{DATASETS_PATH}/{source_name}.csv",
                         index_col=0,
                         parse_dates=['post_time', 'parse_time'])

    
    def check_new_articles(self, nurls, proxies=proxies):
        '''Проверяет, появились ли на ресурсе новые публикации,
        и если появились, заносит их в план на скачивание.'''
        all_new_urls = nurls[self.source_name]
        page_count = 1
        source = self.source

        attempts_counts = 0
        while True:
            try:
                page_url = source['first_page_url'].replace('1', str(page_count))
                page = requests.get(page_url,
                                    headers=headers,
                                    proxies=proxies,
                                    stream=False)
                soup = BeautifulSoup(page.text, 'html.parser')
                urls = {url['href'] for url in soup.find_all(*source['find_all_args'])}
                if not urls:
                    # the thread processes the case when captcha and no urls
                    print("Changing proxy because of no urls or CAPTCHA...")
                    attempts_counts += 1
                    if attempts_counts >= 2:
                        print(f"Data parsing of {self.source_name} is stopped at {page_url}.")
                        break
                    proxies = None
                    continue
                old_urls = set(self.df.index.to_list())
                new_urls = urls - old_urls
                all_new_urls = all_new_urls | new_urls
                nurls[self.source_name] = all_new_urls
                if len(new_urls) > 0:
                    print(f"{len(new_urls)} urls are collected from {page_url}")
                if (len(new_urls) == 0) or (len(urls) == 0):    
                    print(f"{len(all_new_urls)} new urls are saved for {source['base_url']}")
                    break
                else:
                    page_count += 1
            except ConnectionError:
                print("Problems with connection...")
                continue
        with open('not_processed_urls.pickle', 'wb') as f:
            pickle.dump(nurls, f)
        return nurls


    def save_new_data(self, url, data, full_text):
        '''Сохраняет новые данные в датафрейм, а текст -- в отдельный файл'''
        data['filename'] = str(uuid.uuid5(uuid.NAMESPACE_DNS, url))
        new_row = pd.DataFrame(data, index=[url])
        if url not in self.df:
            self.df = self.df.append(new_row)
        else:
            self.df.iloc[url] = new_row
        
        filepath = f"{DATASETS_PATH}/{self.source_name}/{data['filename']}"
        if not os.path.exists(filepath):
            with open(filepath, 'w') as f:
                full_text = re.sub('\n+', '\n\n', full_text).strip()
                f.write(full_text)
                #print(f"File {self.source_name}/{data['filename']} is saved.")
        else:
            pass
            #print(f"File {self.source_name}/{data['filename']} is already exist.")
            
    def save_df(self):
        self.df.to_csv(f"{DATASETS_PATH}/{self.source_name}.csv")
            
    def concat_csv(self):
        '''
        Соединяет старый csv и тот, что получен из новых данных.
        '''
        pass
    

class HabrParser(Parser):
    '''Парсер для сайта habr.com'''
    def update(self, url):
        '''Скачивает или обновляет данные о статье.'''
        data = dict()
        page = requests.get(url, headers=headers, stream=False)
        soup = BeautifulSoup(page.text, 'html.parser')
        data['title'] = soup.select_one('h1').text.strip()
        try:
            date_time = soup.select_one('.post__time').get('data-time_published')
            data['post_time'] = datetime.datetime.strptime(date_time, '%Y-%m-%dT%H:%MZ')
        except AttributeError:
            date_time = None
        try:
            data['views_num'] = k_to_num(soup.select_one('.post-stats__views-count').text.strip())
        except AttributeError:
            data['views_num'] = 0
        try:
            data['likes_num'] = k_to_num(soup.select_one('.js-post-vote').text.strip().replace('–', '-'))
        except AttributeError:
            data['likes_num'] = 0
        try:
            data['favs_num'] = k_to_num(soup.select_one('.bookmark__counter').text)
        except AttributeError:
            data['favs_num'] = 0
        try:
            data['comments_num'] = k_to_num(soup.select_one('.post-stats__comments-count').text)
        except AttributeError:
            data['comments_num'] = 0
        data['parse_time'] = datetime.datetime.now()
        try:
            full_text = soup.select_one('.post__body_full').text
        except AttributeError:
            full_text = ""
        self.save_new_data(url, data, full_text)


class TprogerParser(Parser):
    '''Парсер для сайта tproger.ru'''
    def update(self, url):
        '''Скачивает или обновляет данные о статье.'''
        data = dict()
        try:
            driver = webdriver.Chrome('/usr/bin/chromedriver')
            driver.get(url)
            page = driver.page_source
            element_present = EC.presence_of_element_located((By.CSS_SELECTOR, '.post-views-count'))
            WebDriverWait(driver, timeout=10).until(element_present)
        except TimeoutException:
            print("Timed out waiting for page to load")
        finally:
            driver.quit()

        soup = BeautifulSoup(page, 'html.parser')
        data['title'] = soup.h1.text.strip()
        data['post_time'] = datetime.datetime.fromisoformat(soup.select_one('time')['content'])
        paragraphs = soup.find_all('div', {'class':'entry-content'})[0].find_all('p')
        try:
            data['summary'] = paragraphs[0].text
        except IndexError:
            data['summary'] = ''
        try:
            data['views_num'] = soup.select_one('.post-views-count').text
        except AttributeError:
            data['views_num'] = 0
        data['parse_time'] = datetime.datetime.now()
        full_text = "\n\n".join(p.text for p in paragraphs[:-1])
        self.save_new_data(url, data, full_text)    
        
        
class ParserComposer:
    def __init__(self):
        self.parsers = [HabrParser('habr'),
                        TprogerParser('tproger')] 
        with open('not_processed_urls.pickle', 'rb') as f:
            self.nurls = pickle.load(f)
        
    def load_not_processed_articles(self):
        '''Загружает и обрабатывает статьи что не были обработаны
        по каким-то причинам ранее (обрыв сигнала, вызов исключения и т.д.)'''
        
        pairs = [(key, item) for key in self.nurls for item in self.nurls[key]]
        random.shuffle(pairs)
        
        try:
            for pair in tqdm(pairs):
                try:
                    parser = [parser for parser in self.parsers if parser.source_name == pair[0]][0]
                    parser.update(pair[1])
                except ConnectionError:
                    continue
        finally:
            for parser in self.parsers:
                parser.save_df()
                self.nurls[parser.source_name] -= set(parser.df.index)
            with open('not_processed_urls.pickle', 'wb') as f:
                pickle.dump(self.nurls, f)
    
    
    def update_not_old_articles(self):
        for parser in self.parsers:
            n = datetime.datetime.now()
            p = parser.df.post_time.apply(lambda x: x.replace(tzinfo=None))
            #! Второе условие должно быть по парсингу, а не дате выхода статьи
            condition = ((n - p) <= datetime.timedelta(days=30)) & ((n - p) >= datetime.timedelta(days=2))
            parser.not_old = set(parser.df[condition].index)
        pairs = [(parser.source_name, url) for parser in self.parsers for url in parser.not_old]
        random.shuffle(pairs)
        try:
            for pair in tqdm(pairs):
                try:
                    parser = [parser for parser in self.parsers if parser.source_name == pair[0]][0]
                    parser.update(pair[1])
                except ConnectionError:
                    continue
        finally:
            for parser in self.parsers:
                parser.save_df()
        
    
    def scheduler(self):
        '''
        3) загружаем данные новых статей
        -> 4) обновляем число просмотров статей, вышедших в последний месяц (post_time)
        5) обновляем число просмотров статей, проверявшихся > месяца назад (parse_time), не относящихся к п. 2-3
        '''
        pass
    
    def pipeline(self):
        '''Сортирует задачи таким образом, чтобы
        равномерно распределить запросы к внешним ресурсам.
        Проверяем, вышли ли новые статьи. Планируем загрузки. Обновляем датасеты.'''
        print("Load articles not processed before...")
        self.load_not_processed_articles()
        for parser in self.parsers:
            print("Check new articles...")
            self.nurls = parser.check_new_articles(self.nurls)
            print("Update data for not old articles...")
            self.update_not_old_articles()
            
#         with open('not_processed_urls.pickle', 'wb') as f:
#             pickle.dump(self.nurls, f)


In [7]:
composer = ParserComposer()
composer.pipeline()

FileNotFoundError: [Errno 2] No such file or directory: '/home/leo/DATASETS/habr.csv'

In [19]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

class HabrSpider(scrapy.Spider):
    name = "habr.com"
    start_urls = ['https://habr.com/ru/hubs/']

    def parse(self, response):
        for hub_url in response.css('.list-snippet__title-link'):
            yield {
                'hub_url': hub_url.css('a::attr(href)').getall()
            }

process = CrawlerProcess(get_project_settings())
process.crawl(HabrSpider)
#process.start()
#process.stop()

2021-04-20 21:43:42 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: scrapybot)
2021-04-20 21:43:42 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.8.5 (default, Jan 27 2021, 15:41:15) - [GCC 9.3.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.4.7, Platform Linux-5.4.0-72-generic-x86_64-with-glibc2.29
2021-04-20 21:43:42 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-04-20 21:43:42 [scrapy.crawler] INFO: Overridden settings:
{}
2021-04-20 21:43:42 [scrapy.extensions.telnet] INFO: Telnet Password: 7dc510bcf332c96b
2021-04-20 21:43:42 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2021-04-20 21:43:42 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloade

In [46]:
import re
import datetime

full_months = ['января', 'февраля', 'марта',
               'апреля', 'мая', 'июня',
               'июля', 'августа', 'сентября',
               'октября', 'ноября', 'декабря']

short_months = [month_name[:3] for month_name in full_months]
sm = '|'.join(short_months)


def str_to_datetime(s):
    '''Take string with date in russian and return datetime object'''
    sentence = s.strip().lower().split()
    now = datetime.datetime.now()
    try:
        year = int(re.search(r'\d{4}', s).group(0))
    except AttributeError:
        year = now.year
    try:
        month = re.search(f'{sm}', s).group(0)
        month = short_months.index(month) + 1
        day = int(
            re.search(r'(?<!\d{2})\d{1,2}(?=\s)(?! дня|\d+ |:\d{2})', s).group(0))
    except AttributeError:
        if re.match(r'вчера', s):
            yesterday = (now - datetime.timedelta(days=1))
            month = yesterday.month
            day = yesterday.day
        elif re.match(r'позавчера', s):
            yesterday2 = (now - datetime.timedelta(days=2))
            month = yesterday2.month
            day = yesterday2.day
        elif re.match(r'завтра', s):
            tomorrow = (now - datetime.timedelta(days=2))
            month = tomorrow.month
            day = tomorrow.day
        else:
            month = now.month
            day = now.day
    try:
        hour = int(re.search(r'\d{2}(?=:\d{2})', s).group(0))
        minute = int(re.search(r'(?<=\d{2}:)\d{2}', s).group(0))
    except AttributeError:
        if re.match(r'^\d+(?= мин)', s):
            t = now - \
                datetime.timedelta(minutes=int(
                    re.match(r'\d+(?= мин)', s).group(0)))
        elif re.match(r'^\d+(?= час)', s):
            t = now - \
                datetime.timedelta(
                    hours=int(re.match(r'\d+(?= час)', s).group(0)))
        else:
            t = now
        hour = t.hour
        minute = t.minute
    return datetime.datetime(year,
                             month,
                             day,
                             hour,
                             minute)

In [ ]:
def habr_prepare(path=f'{DATASETS_PATH}/habr.json'):
    #df_habr['filename'] = df['url'].apply(lambda x: str(uuid.uuid5(uuid.NAMESPACE_DNS, x)))
    df_habr = pd.read_json(path).set_index('url')
    df_habr = df_habr[~df_habr.index.duplicated(keep='first')]
    df_habr.post_time = df_habr.post_time.apply(str_to_datetime)
    df_habr.post_time = pd.to_datetime(df_habr.post_time)
    df_habr.parse_time = pd.to_datetime('today')
    df_habr['parse_time'] = pd.to_datetime(datetime.date.today())
    df_habr.likes_num = df_habr.likes_num.apply(lambda x: x.replace('–', '-')).apply(int)
    df_habr.views_num = df_habr.views_num.apply(lambda x: x.replace(',', '.').replace('k', 'e+3')).apply(float).apply(int)
    df_habr.comments_num = df_habr.comments_num.fillna(0).apply(int)
    df_habr.summary = df_habr.summary.apply(lambda x: re.sub(r'[\n\r\s]{2,}', '\n', x.replace('Читать дальше →', '').strip()))
    df_habr['source'] = 'habr'
    return df_habr

df_habr = habr_prepare()

,title,comments_num,summary,post_time,fulltext,parse_time,source
url,,,,,,,
https://dev.by/news/avstraliya-hochet-oshtrafovat-facebook-na-350-mlrd-za-skandal-s-cambridge-analytica,Австралия хочет оштрафовать Facebook на $350 м...,0,Австралийские правозащитники подали в суд на F...,2020-03-09 16:27:00,Австралийские правозащитники подали в суд на F...,2021-04-26,devby
https://dev.by/news/zagruzki-google-pay-v-mire-vyrosli-v-dva-raza,Загрузки Google Pay в мире выросли в два раза,0,Google Pay второй месяц подряд является самым ...,2020-03-30 15:15:00,Google Pay второй месяц подряд является самым ...,2021-04-26,devby
https://dev.by/news/godel-technologies-ispolnyaet-zhelaniya-na-great-british-festival,Godel Technologies исполняет желания на Great ...,0,,2018-06-19 16:15:00,"С фестивальной субботы прошло уже два дня, а с...",2021-04-26,devby
https://dev.by/news/There-and-Back-Again,"Девять лет проработал менеджером, а потом верн...",0,,2019-08-23 09:15:00,Андрей Ворошков начинал программистом на БелАЗ...,2021-04-26,devby
https://dev.by/news/belorusskoe-prilozhenie-dlya-perevoda-sketchei-v-makety-vyshlo-na-product-hunt,Белорусское приложение для перевода скетчей в ...,0,,2019-08-14 10:50:00,Мобильное приложение для мгновенной конвертаци...,2021-04-26,devby
...,...,...,...,...,...,...,...
https://dev.by/news/haiv-prodzhekt-dfr-i-kgb,"Директора «Хайв Проджект» (MolaMola, Ulej) уве...",0,"Директора «Хайв Проджект» (MolaMola.by, Ulej.b...",2020-08-04 22:01:00,"Директора «Хайв Проджект» (MolaMola.by, Ulej.b...",2021-04-26,devby
https://dev.by/news/dmitriev-salary,"Штаб Дмитриева запустил сервис, который считае...",0,Кандидат в президенты Андрей Дмитриев сообщил ...,2020-07-31 16:04:00,Кандидат в президенты Андрей Дмитриев сообщил ...,2021-04-26,devby
https://dev.by/news/hacaton-idea-vote,Что придумали 200+ разработчиков на хакатоне п...,0,18-19 июля в Минске прошёл хакатон Social Tech...,2020-07-21 17:16:00,18-19 июля в Минске прошёл хакатон Social Tech...,2021-04-26,devby
